# README
The folder containing this notebook also contains data used to produce many of the figures in Hafen+2018. This notebook clarifies what each data product is, and provides example code for reading in the data.

**If you have any questions or would like data not available here,
please contact Zach Hafen at zhafen@u.northwestern.edu.**

In [1]:
import h5py
import numpy as np

## Mass Budget Data

Files that start with `halo_mass_content_FIRE2` contain data for Figures 2 and 3.

* All masses are in units of $M_\odot$.
* Snapshots (600, 465, 172) correspond to redshifts (0, 0.25, 2) respectively.
* If you forget what redshift a HDF5 file is associated with, that's also stored in the attributes, `f.attrs['redshift']`.
* If you want to identify what sim a value is associated with, use e.g. `f['M']['total']['name'][...]`.
* In Figures 1 and 2 the x-data is the halo mass, `f['M']['total']['m_vir'][...]`, but you could also use the stellar mass, `f['M']['stars']['m_gal'][...]`.
* The FIRE-1 datasets are currently incomplete, but may be completed in the future.

In [2]:
# Load an example file
f = h5py.File( 'halo_mass_content_FIRE2_snum600.hdf5', 'r' )

In [3]:
# Baryon Masses:
baryon_masses = {
    'Total halo baryon mass': f['M']['total']['m_halo'][...],
    'Total CGM gas mass': f['M']['gas']['m_cgm'][...],
    'Total galaxy baryon mass': f['M']['total']['m_gal'][...],
    'Total galaxy-halo interface gas mass': f['M']['gas']['m_interface'][...],
}
# To make Figure 1, divide these by the cosmological baryon budget, f_b*M_h,
total_budget = f['M']['total']['m_budget_halo'][...]

In [4]:
# Metal Masses:
metal_masses = {
    'Total halo metal mass': f['enriched_metal_mass']['total']['m_halo'][...],
    'Total CGM gas metal mass': f['enriched_metal_mass']['gas']['m_cgm'][...],
    'Total galaxy metal mass': f['enriched_metal_mass']['total']['m_gal'][...],
    'Total galaxy-halo interface gas metal mass': f['enriched_metal_mass']['gas']['m_interface'][...],
}
# To make Figure 2, divide these by our estimate of the metal budget, y_box * M_star:
# Here y_box is the yield for our simulation volume
# (total mass of metals in the volume divided by total mass of stars in the volume)
# and M_star is the mass of the central galaxy.
metal_budget = f['enriched_metal_mass']['total']['m_budget_halo'][...]

In [5]:
# Other data, e.g. temperatures in Kelvin
halo_mass_above_m12 = f['M']['total']['m_vir'][...] > 1e12
temps_m12 = f['T']['T_CGM_median'][...][halo_mass_above_m12]
print(
    'Median temperature of M_h > 1e12 Msun halos at z = {:.3g} is {:.2g} K'.format(
        f.attrs['redshift' ],
        np.median( temps_m12 ),
    )
)

Median temperature of M_h > 1e12 Msun halos at z = 0 is 4.4e+05 K


## CGM Mass Contributed by Different Origins

Files matching `cgm_origins*frac*` contain data for Figures 9 and 10.

In [6]:
f = h5py.File( 'cgm_origins_mass_frac_snum465.hdf5' )

In [7]:
# For example, this is the fraction of CGM mass contributed by IGM accretion.
f['is_CGM_IGM_accretion'][...]

array([0.58984572, 0.9212662 , 0.34212539, 0.6434502 , 0.38089553,
       0.60524637, 0.4576776 , 0.5559122 , 0.57855153, 0.45636287,
       0.54578125, 0.63538939, 0.70268017, 0.72218126, 0.60755271,
       0.62912339, 0.70187688, 0.54861581, 0.51560831, 0.4162769 ,
       0.72024947])

In [8]:
# The simulations associated with each value are in this array
f['name'][...]

array([b'm10q', b'm10v', b'm11e_md', b'm10z', b'm11v', b'm11q', b'm10y',
       b'm12r_md', b'm12f', b'm11i_md', b'm11h_md', b'm12m', b'm11a',
       b'm12i', b'm11c', b'm11b', b'm12b_md', b'm12z_md', b'm12w_md',
       b'm11d_md', b'm12c_md'], dtype='|S7')

## Azimuthal Profiles

Files matching `cgm_origins_azimuthal_dep` contain data for Figures 16 and 17.

In [9]:
f = h5py.File( 'cgm_origins_azimuthal_dep_snum465.hdf5' )

In [10]:
# For example the following produces the green "central-wind" line
# in the top-left panel of Figure 17

f['cold']['is_CGM_wind']['45.0']['m12f'] [...]

array([0.9460289 , 0.895894  , 0.8658334 , 0.8524243 , 0.8240076 ,
       0.82066286, 0.8222937 , 0.82958454, 0.8130362 , 0.81402725,
       0.8139105 , 0.780781  , 0.75863016, 0.75647455, 0.75206417,
       0.7147166 , 0.6774078 , 0.60601795, 0.6542289 , 0.6799292 ,
       0.6525052 , 0.6365151 , 0.61208117, 0.6297597 , 0.5360156 ,
       0.53651804, 0.6601969 , 0.63506585, 0.6955709 , 0.7201427 ,
       0.7711705 ], dtype=float32)

In [11]:
# These should be combined with the following array, r_rvir, for
# the radial distance in units of r_vir (x-axis)

bin_edges = np.linspace( 0.1, 1., 32 )
r_rvir = bin_edges[:-1] + 0.5 * ( bin_edges[1] - bin_edges[0] ) 

## Radial Profiles

Files matching `radial_profiles` contain data for Figures 15 and 19.

In [12]:
f = h5py.File( 'radial_profiles_snum465.hdf5' )

In [13]:
f.keys()

KeysView(<HDF5 file "radial_profiles_snum465.hdf5" (mode r+)>)

These files contain data on the total Hydrogen density as a function of radius (`density`) and the metallicity as a function of radius (`metallicity`; in units of Z_sun = 0.02).

In [14]:
# For example, this is the volume weighted mean density for Mh = 1e12 Msun progenitors
# for all gas in the CGM (i.e. without separating out origins)
f['density']['m12']['is_in_CGM']['volume_weighted_mean'][...]

array([9.30533099e-04, 6.41579961e-04, 4.95163757e-04, 4.43630537e-04,
       3.60100384e-04, 2.92762527e-04, 2.43099642e-04, 2.30154656e-04,
       2.15201859e-04, 1.94786931e-04, 1.82113649e-04, 1.63687796e-04,
       1.42777198e-04, 1.19645404e-04, 1.07284715e-04, 1.00984534e-04,
       9.38372980e-05, 8.91469974e-05, 9.23640469e-05, 9.11527480e-05,
       9.04047979e-05, 7.52967836e-05, 6.94376596e-05, 6.43239582e-05,
       5.72735503e-05, 5.16483200e-05, 4.76873528e-05, 4.42105642e-05,
       4.13986095e-05, 4.14022385e-05, 3.71555834e-05, 3.47797992e-05,
       3.30401052e-05, 3.13601432e-05, 2.92170023e-05, 2.87635497e-05,
       3.05915059e-05, 2.92891661e-05, 2.51178484e-05, 2.27307994e-05,
       2.13850536e-05, 2.03244352e-05, 1.98040776e-05, 1.87235000e-05,
       1.78731149e-05, 1.73146049e-05, 1.65467400e-05, 1.59499400e-05,
       1.54404184e-05, 1.51493965e-05, 1.46618966e-05, 1.40681888e-05,
       1.35931905e-05, 1.30576148e-05, 1.25998715e-05, 1.22427242e-05,
      

In [15]:
# These are the corresponding x-values (in units of Rvir) for the above array
f['density']['m12']['is_in_CGM']['x_values'][...]

array([0.1071567 , 0.12144219, 0.13572768, 0.15001317, 0.16429867,
       0.17858416, 0.19286965, 0.20715515, 0.22144064, 0.23572613,
       0.25001162, 0.26429712, 0.27858261, 0.2928681 , 0.3071536 ,
       0.32143909, 0.33572458, 0.35001007, 0.36429557, 0.37858106,
       0.39286655, 0.40715205, 0.42143754, 0.43572303, 0.45000852,
       0.46429402, 0.47857951, 0.492865  , 0.5071505 , 0.52143599,
       0.53572148, 0.55000697, 0.56429247, 0.57857796, 0.59286345,
       0.60714895, 0.62143444, 0.63571993, 0.65000542, 0.66429092,
       0.67857641, 0.6928619 , 0.7071474 , 0.72143289, 0.73571838,
       0.75000387, 0.76428937, 0.77857486, 0.79286035, 0.80714585,
       0.82143134, 0.83571683, 0.85000232, 0.86428782, 0.87857331,
       0.8928588 , 0.9071443 , 0.92142979, 0.93571528, 0.95000077,
       0.96428627, 0.97857176, 0.99285725])

In [16]:
# As mentioned above, the redshift is stored in the attributes
f.attrs['redshift']

0.25099599